In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

c:\Users\nolan\anaconda3\envs\sp25_taxi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from datetime import datetime, timedelta
import pandas as pd

# Get the current datetime64[us, Etc/UTC]
current_date = pd.Timestamp.now(tz="US/Eastern")
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-02 20:50:19,786 INFO: Initializing external client
2025-03-02 20:50:19,786 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 20:50:20,926 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Fetching data from 2025-02-01 20:50:19.786187-05:00 to 2025-03-02 19:50:19.786187-05:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.54s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-02 02:00:00+00:00,2,0
1,2025-02-02 03:00:00+00:00,2,0
2,2025-02-02 04:00:00+00:00,2,0
3,2025-02-02 05:00:00+00:00,2,0
4,2025-02-02 06:00:00+00:00,2,0
...,...,...,...
177915,2025-03-02 20:00:00+00:00,263,130
177916,2025-03-02 21:00:00+00:00,263,93
177917,2025-03-02 22:00:00+00:00,263,114
177918,2025-03-02 23:00:00+00:00,263,114


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177920 entries, 0 to 184063
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         177920 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  177920 non-null  int32                  
 2   rides               177920 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.1 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_convert("US/Eastern")

In [8]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-01 21:00:00-05:00,2,0
1,2025-02-01 22:00:00-05:00,2,0
2,2025-02-01 23:00:00-05:00,2,0
3,2025-02-02 00:00:00-05:00,2,0
4,2025-02-02 01:00:00-05:00,2,0
...,...,...,...
177915,2025-03-02 15:00:00-05:00,263,130
177916,2025-03-02 16:00:00-05:00,263,93
177917,2025-03-02 17:00:00-05:00,263,114
177918,2025-03-02 18:00:00-05:00,263,114


In [9]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177920 entries, 0 to 184063
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                     
---  ------              --------------   -----                     
 0   pickup_hour         177920 non-null  datetime64[us, US/Eastern]
 1   pickup_location_id  177920 non-null  int32                     
 2   rides               177920 non-null  int32                     
dtypes: datetime64[us, US/Eastern](1), int32(2)
memory usage: 4.1 MB


In [10]:
from src.data_utils import transform_ts_data_info_features_and_target

features, targets = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

In [11]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2025-02-20 04:00:00,217
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2025-03-02 01:00:00,65
2,0,13,7,0,0,0,0,0,0,0,...,10,45,0,0,0,0,0,0,2025-02-04 16:00:00,261
3,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,2025-02-18 10:00:00,69
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2025-02-17 21:00:00,155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,2025-02-05 22:00:00,168
252,2,0,0,0,0,0,5,0,0,0,...,0,0,0,18,7,6,0,0,2025-02-08 22:00:00,74
253,0,0,0,0,0,0,0,9,0,0,...,0,0,0,0,0,1,0,0,2025-02-25 09:00:00,80
254,0,0,0,0,0,0,28,0,0,0,...,0,0,0,0,0,0,0,143,2025-02-19 14:00:00,79


In [12]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Columns: 674 entries, rides_t-672 to pickup_location_id
dtypes: datetime64[ns](1), int32(1), int64(672)
memory usage: 1.3 MB


In [13]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='US/Eastern')
features = load_batch_of_features_from_store(current_date)

2025-03-02 20:51:48,934 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 20:51:48,950 INFO: Initializing external client
2025-03-02 20:51:48,950 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 20:51:49,854 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Fetching data from 2025-02-01 20:51:48.934408-05:00 to 2025-03-02 19:51:48.934408-05:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.38s) 


In [14]:
current_date

Timestamp('2025-03-02 20:51:48.934408-0500', tz='US/Eastern')

In [15]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2025-03-02 02:00:00,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2025-03-02 02:00:00,3
2,0,0,0,0,0,0,0,0,0,0,...,16,4,11,12,27,16,16,4,2025-03-02 02:00:00,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,2025-03-02 02:00:00,6
4,0,0,0,0,0,0,0,0,0,0,...,6,9,2,3,3,4,4,3,2025-03-02 02:00:00,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,2025-03-02 02:00:00,259
252,0,0,0,0,0,0,0,0,0,0,...,3,1,0,1,3,4,1,1,2025-03-02 02:00:00,260
253,0,0,0,0,0,0,0,0,0,0,...,50,42,46,50,57,54,37,23,2025-03-02 02:00:00,261
254,0,0,0,0,0,0,0,0,0,0,...,139,110,99,85,112,103,95,58,2025-03-02 02:00:00,262


In [16]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-02 20:52:36,039 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 20:52:36,048 INFO: Initializing external client
2025-03-02 20:52:36,048 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 20:52:36,858 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635


In [17]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [18]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,5.0
3,6,0.0
4,7,1.0
...,...,...
251,259,0.0
252,260,0.0
253,261,17.0
254,262,42.0


In [19]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 249, 161, 236, 142, 237,  79, 230,  48, 163])